## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Вводные данные:**

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортирую библиотеку
import pandas as pd
#считываю файл
df = pd.read_csv('/datasets/data.csv')
df.to_csv('data.csv', index=False)
# вызываю первые 10 строк для визуального анализа
df.head(40)
#вызываю общую информацию
#df.info()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод** 

В таблице представленно 21525 строк
Так же таблице 12 столбцов. Тип данных в столбцах — object,int64,float64

Согласно документации к данным:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

В названиях колонок нарушений не видно.

 - В колонке ['days_employed'] есть отрицательные заначения и пропуски
 - В колонке ['total_income'] есть пропуски
 
 - Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [9]:
# проверяю название столбцов
df.columns
# подсчёт пропусков
df.isna().sum()


#замена пропусков в столбцах (days_employed,total_income) на 0.
columns_to_replace = ['days_employed','total_income']
for i in columns_to_replace:
    df[i] = df[i].fillna(0)

# проверка 
df.isna().sum()

# переведём все отрицательные значения столбца(days_employed) в положительные методо abs()
df['days_employed'] = abs(df['days_employed'])
df['days_employed']
# переведём все отрицательные значения столбца(children) в положительные методо abs()
df['children'] = abs(df['children'])
df['children']
# в столбце (gender) есть опечатка XNA
df.loc[df.loc[:, 'gender'] == 'XNA']
# переведём XNA в F
df.loc[:, 'gender'] = df.loc[:, 'gender'].replace('XNA','F')
# проверка 
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод** 

Убедились, что в таблице не осталось пропусков. Для этого ещё раз посчитали пропущенные значения.
    Пропуски связаны из-за: двух данных столбцов (days_employed,total_income) взаимо связанны между собой и при пропуске значения в колонке Ежемесяный доход сразу обнуляется трудовой стаж и наоборот.

### Замена типа данных

In [10]:
# В таблице имеем два стобца,где имеется тим данных float (days_employed,total_income), заменим на тип данных (int) с помощью astype()
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
#выводим для проверки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Заменили данные на int для более корректной работы

### Обработка дубликатов

In [11]:
#поиск дубликатов
df.duplicated().sum()
df.duplicated().value_counts()

#обнаруженно 54 дубликата
sorted(df['education'].
       unique())

#приводим все значения к нижнему регисру методом str.lower()
df['education'] = df['education'].str.lower()

#обнаруженно 71 дубликат
df.duplicated().value_counts()

#удаляем дубликаты
df = df.drop_duplicates().reset_index(drop=True)

#проверяем
df.duplicated().sum()



0

**Вывод**

Нашли дубликаты как явные так и неявные, удалили.

### Лемматизация

In [12]:
#выделены леммы в значениях столбца с целями получения кредита
#выведем уникальные значения столбца (purpose)
df['purpose'].value_counts()
#импортируем библиотеки, для стемминга и леманитизации family_status_id
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

m = Mystem()
pur = []
for lm in df['purpose']:
    lemma = m.lemmatize(lm)
    pur.extend(lemma) #методом extend() ,объединям две таблицы
Counter(pur)

#перепроверим и создадим 4 позиции которые будут отвечать за (авто,свадьбу,недвижк/жильё,образование) 
car = 0
sv = 0
ed = 0
hom = 0
for jop in df['purpose']:
    if 'авт' in jop:
        car += 1
    elif 'сва' in jop:
        sv += 1
    elif 'обр' in jop:
        ed += 1
    elif 'нед' in jop:
        hom += 1
    else: 
        hom += 1

hom
total = car + sv + ed + hom
total


21454

**Вывод**

проведя лемматизацию и посчитав вручную через циклы, разделили потребности кретидов на 4 группы:
авто - 4306
свадьба - 2324
образование - 4013
жильё/недвижка - 10811
тотал - 21454

### Категоризация данных

In [13]:
# Создадим фукцию и присвоим к 4 столбца purpose категориям свою цифру 1-авто,2-свадьба,3-образование,4-недвиж/жильё
def alert_group(cat):
    if 'авто' in cat:
        return 1
    elif 'сва' in cat:
        return 2
    elif 'обр' in cat:
        return 3
    else:
        return 4
alert_group('автомобиль')
# Создадим новый столбец (pur_cat)
df['pur_cat'] = df ['purpose'].apply(alert_group)

#категоризация по столбцу (education)
df.groupby(['education','education_id']).count().reset_index()
#категоризация по столбцу (family_status)
df.groupby(['family_status','family_status_id']).count().reset_index()

#Категоризируем по доходу, посмотрим среднее значение
df['total_income'].mean()
# Ср. знач 151010
#Сделаем 3 класса (Стандартный до 50,средний до 200,богатый свыше 200)
def income_cat(cat):
    if cat <= 50000:
        return 'стандартный'
    elif cat <= 200000:
        return 'средний'
    else:
        return 'богатый'
# Создаём новый столбец
df['total_cat'] = df['total_income'].apply(income_cat)
df
# Категоризируем детей в 3 категории (Нет детей,1ребёнк и мноогодетные)
def chil_cat(cat):
    if cat == 0:
        return 'нет детей'
    elif cat <= 1:
        return '1 ребёнок'
    else:
        return 'многодетные'

chil_cat(3)  
#Создаём новый столбец с категорией дети
df['children_cat'] = df['children'].apply(chil_cat)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,pur_cat,total_cat,children_cat
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,4,богатый,1 ребёнок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,1,средний,1 ребёнок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,4,средний,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3,богатый,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,2,средний,нет детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,4,богатый,1 ребёнок
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,1,средний,нет детей
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,4,средний,1 ребёнок
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,1,богатый,многодетные


**Вывод**

Категоризавали данные по столбцам с доходом и категориями взятия кредита. Столбцы с семейным положением и образованием уже предоставленны.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [1]:
debt_children = pd.DataFrame()
debt_children['count_children'] = df.groupby('children_cat')['debt'].count()
debt_children['count_children']
debt_children['sum_children'] = df.groupby('children_cat')['debt'].sum()
debt_children['sum_children']
debt_children['result_children'] = debt_children['sum_children'] / debt_children['count_children'] 
debt_children['result_children']
debt_children.sort_values('result_children', ascending = False)



NameError: name 'pd' is not defined

**Вывод**

Клиенты у кого есть более 1 ребёнка наиболее склонны к просрочке нежели бездетные.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = df.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = df.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод**

Люди у кого фоициально нет отношений или брака , склонны к просрочке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
debt_total_income = pd.DataFrame()
debt_total_income['sum_total'] = df.groupby('total_cat')['debt'].sum()
debt_total_income['count_total'] = df.groupby('total_cat')['debt'].count()
debt_total_income['result_total'] = debt_total_income['sum_total'] / debt_total_income['count_total']
debt_total_income.sort_values('result_total',ascending = False)

,sum_total,count_total,result_total
total_cat,,,
средний,1190,13913,0.085532
стандартный,193,2475,0.077980
богатый,358,5066,0.070667


**Вывод**

Клиенты с наиболее высокой з/п деют просрочек по кредиту горяздо меньше, стандартного и среднего класса

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = df.groupby('pur_cat')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = df.groupby('pur_cat')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
pur_cat,,,
1,403,4306,0.093590
3,370,4013,0.092200
2,186,2324,0.080034
4,782,10811,0.072334


**Вывод**

Клиенты берущие кредиты на жильё/недвижимость, наиболее ответсвенно относятся к погашению. нежели клиенты взявшие кредит на авто

## Шаг 4. Общий вывод

Изначально у нас стояло 4 впороса-гипотезы:
Есть ли зависимость между наличием детей и возвратом кредита в срок?
Ответ: Да, чем меньше детей,тем меньше просрочек по кредиту

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Ответ: Да,находится человек в официальном браке или нет показываёт своё влияние.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Ответ: Да, чем состоятельнее человек, тем проблем с погашением меньше.

Как разные цели кредита влияют на его возврат в срок?
Ответ: Высокая доля просрочки идёт у владельцев автомобилей, владельцы недвижеммости наименее склоны к просрочке

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.